In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from torchvision.transforms import ToTensor
from torchvision import transforms
import torch
import importlib
import random
import json

import cv2
from skimage import io

In [2]:
NUM_CLASSES = 3
HEIGHT = 512
WIDTH = 904

In [3]:
model_file = importlib.import_module("erfnet")
model = model_file.Net(NUM_CLASSES)
model = torch.nn.DataParallel(model).cuda()

In [4]:
model.load_state_dict(torch.load("/home/yerzh/cv/project/model/erfnet_pytorch/save/model_best.pth"))

<All keys matched successfully>

In [5]:
json_path='/home/yerzh/cv/project/train_set/label_data_0531.json'
root_dir='/home/yerzh/cv/project/train_set/'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
op_transforms = transforms.Compose([transforms.ToTensor()])

In [7]:
scores=[]
def accuracy(model, json_path, root_dir, device, op_transforms):
    json_gt = [json.loads(line) for line in open(json_path)]
    for gt in json_gt:
        gt_lanes = gt['lanes']
        y_samples = gt['h_samples']
        raw_file = gt['raw_file']
        img = cv2.imread(root_dir + raw_file)
        gt_lanes_vis = [[(x, y) for (x, y) in zip(lane, y_samples) if x >= 0] for lane in gt_lanes]
        mask = np.zeros_like(img[:,:,0])
        cv2.polylines(mask, np.int32([gt_lanes_vis[0]]), isClosed=False,color=255, thickness=5)
        cv2.polylines(mask, np.int32([gt_lanes_vis[1]]), isClosed=False,color=255, thickness=5)
        im_tensor = torch.unsqueeze(op_transforms(img), dim=0)
        im_tensor = im_tensor.to(device)
        model = model.to(device)
        with torch.no_grad():
            model = model.eval()
            out = model(im_tensor)
        out = out.max(dim=1)[1]
        out_np = out.cpu().numpy()[0].reshape(mask.shape)
        lines = cv2.HoughLinesP(out_np.astype(np.uint8), 1, np.pi/180, 30, maxLineGap=200)
        dmy = out_np.astype(np.uint8).copy()
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(dmy, (x1, y1), (x2, y2), (255, 0, 0), 5)
        start=0
        end=0
        start2=0
        end2=0
        for i in range(mask.shape[0]):
            if len(np.unique(mask[i]))==1:
                continue
            flag=0
            dist=0
            for j in range(2,mask.shape[1]-2):
                if mask[i][j-2]!=0 and mask[i][j-1]!=0 and mask[i][j]!=0 and mask[i][j+1]==0:
                    dist=0
                    start=end
                elif mask[i][j-1]==0 and mask[i][j]!=0 and mask[i][j+1]!=0 and mask[i][j+2]!=0:
                    if flag==1:
                        break
                    flag+=1
                dist+=1
                end+=1
            flag2=0
            dist2=0
            for j in range(2,dmy.shape[1]-2):
                if dmy[i][j-2]!=0 and dmy[i][j-1]!=0 and dmy[i][j]!=0 and dmy[i][j+1]==0:
                    dist2=0
                    start2=end2
                elif dmy[i][j-1]==0 and dmy[i][j]!=0 and dmy[i][j+1]!=0 and dmy[i][j+2]!=0:
                    if flag2==1:
                        break
                    flag2+=1
                dist2+=1
                end2+=1
            if np.abs(dist-dist2)<11 and np.abs(start-start2)<6 and np.abs(end-end2)<6:
                scores.append(1)
            else:
                scores.append(0)
    return np.array(scores).sum()/len(scores)

In [ ]:
acc = accuracy(model, json_path, root_dir, device, op_transforms)

In [26]:
acc

0.88